# Transformers简介
## 1. 模型结构


<img src="img/transformer.jpeg" height="400" width="400" >

transformer 本质上依旧是encoder_decoder框架，所以我们可以拆成2部分来了解它。

In [1]:
import torch
import torch.nn as nn

### 1.1 encoder
encoder 其实是6个相同结构的encodelayer重复，每个encodelayer是由self_attention和feed_forward组成。

In [ ]:
class Self


class EncodeLayer(nn.Module):
    
    def __init__(self, size)
    
    
    
    
class Encoder(nn.Module):
    def __init__(self, N=6):
        super().__init__()
        